---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [101]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [102]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [103]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def states_map(x):
    x_new = list(states.keys())[list(states.values()).index(x)]
    
    return x_new

import re
def get_list_from_txt():
    ut_data = []
    with open('./university_towns.txt', 'r') as f_read:
        for each_line in f_read:
            # remove the '\n'
            each_line = each_line.strip('\n')
            
            # skip the note line with the ':' as the end of line
            if (each_line.endswith(':')):
                continue
            # check if the line is a new state
            is_state = each_line.find('[edit]')
            if is_state > 0:
                # we got a new state
                (current_state_name, extra) = each_line.split('[')
                #print('--> '+ current_state_name)
            else:
                # this is a university town name with the old state name
                # store it
                # removing every character from " (" to the end
                # special line has two region split by '*'
                result = re.split('[(*(]', each_line)

                region_name = result[0]
                #print('-->--> ' + region_name)
                ut_data.append([current_state_name, region_name])
                
                if (len(result) > 2 and each_line.find('*') > 0):
                    region_name = result[2]
                    #print('-->--> ' + region_name)
                    ut_data.append([current_state_name, region_name])
    return ut_data
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # ut_data = [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ]
    ut_data = get_list_from_txt()
    #ut_data = pd.Series(ut_data)
    ut_df = pd.DataFrame(ut_data, columns=["State", "RegionName"])
    
    # map the state name to two letter acronyms
    ut_df['State'] = ut_df['State'].apply(states_map)
    
    return ut_df

df = get_list_of_university_towns()
df.head()

,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo


In [104]:
def get_city_homeprice():
    ct_home_price_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    return ct_home_price_df

ct_home_price_df = get_city_homeprice()
#print(list(ct_home_price_df.columns))
gdp_df = ct_home_price_df.iloc[:, 6:]
#gdp_df.head()
# gdp_df.info()
gdp_df.columns = pd.DatetimeIndex(gdp_df.columns)
gdp_df.head()
gdp_df_Q = gdp_df.resample('Q', axis=1).mean()
gdp_df_Q.fillna(method = 'bfill', inplace=True, axis = 1)
gdp_df_qsum = gdp_df_Q.sum()
type(gdp_df_qsum)
gdp_df_qsum_df = pd.DataFrame(gdp_df_qsum, columns=['GDP'])
gdp_df_qsum_df.head()

,GDP
1996-06-30,1.367418e+09
1996-09-30,1.369793e+09
1996-12-31,1.373754e+09
1997-03-31,1.384613e+09
1997-06-30,1.395653e+09


In [105]:
gdp_df.head()

,1996-04-01 00:00:00,1996-05-01 00:00:00,1996-06-01 00:00:00,1996-07-01 00:00:00,1996-08-01 00:00:00,1996-09-01 00:00:00,1996-10-01 00:00:00,1996-11-01 00:00:00,1996-12-01 00:00:00,1997-01-01 00:00:00,...,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,155000.0,154600.0,154400.0,154200.0,154100.0,154300.0,154300.0,154200.0,154800.0,155900.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,109700.0,109400.0,109300.0,109300.0,109100.0,109000.0,109000.0,109600.0,110200.0,110800.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,50000.0,49900.0,49600.0,49400.0,49400.0,49300.0,49300.0,49400.0,49700.0,49600.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,87200.0,87700.0,88200.0,88400.0,88500.0,88900.0,89400.0,89700.0,90100.0,90700.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


* A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A recession bottom is the quarter within a recession which had the lowest GDP.
* A university town is a city which has a high percentage of university students compared to the total population of the city.

In [106]:
# define a dictionary to store the recession info
recession_info = {'start':[], 
                 'bottom':[],
                 'end':[]}

# this quarter compare to last one, if Q2 < Q1, then the increase is False
gdp_df_qsum_df['Increase'] = True
for i in range(1, len(gdp_df_qsum_df)-1):
    if gdp_df_qsum_df['GDP'].iloc[i] <= gdp_df_qsum_df['GDP'].iloc[i-1]:
        gdp_df_qsum_df['Increase'].iloc[i] = False

# without list , all the value in Increase would be NAN
#gdp_df_qsum_df['Increase'] = list(gdp_df_qsum_df.reset_index()['GDP'] < gdp_df_qsum_df_cp.reset_index()['GDP'])
#gdp_df_qsum_df['Increase'].iloc[60]
#gdp_df_qsum_df.iloc[1]

# go through the data frame to get recession info
# find the recession start
for i in range(0, len(gdp_df_qsum_df)-2):
    if (gdp_df_qsum_df['Increase'].iloc[i] == False and gdp_df_qsum_df['Increase'].iloc[i+1] == False):
        # we got recession start
        #time = gdp_df_qsum_df['GDP'].index[3]
        recession_start = gdp_df_qsum_df.iloc[i].name
        #print('the recession start at {}'.format(recession_start))
        recession_info['start'] = [i, recession_start]
        print('recession start quarter:{}, index:{}'.format(recession_info['start'][1], recession_info['start'][0]))
        break
        
    
# find the recession end
for i in range(recession_info['start'][0], len(gdp_df_qsum_df)-2):
    if (gdp_df_qsum_df['Increase'].iloc[i] == True and gdp_df_qsum_df['Increase'].iloc[i+1] == True):
        recession_info['end'] = [i, gdp_df_qsum_df.iloc[i].name]
        print('recession end quarter:{}, index:{}'.format(recession_info['end'][1], recession_info['end'][0]))
        break

recession_df = gdp_df_qsum_df[recession_info['start'][0]:recession_info['end'][0]+1]

bottom = recession_df[recession_df['GDP'] == recession_df['GDP'].min()]
#print(type(bottom))
recession_info['bottom'] = [0, bottom.index[0]]
        

C:\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


recession start quarter:2006-12-31 00:00:00, index:42
recession end quarter:2012-06-30 00:00:00, index:64


In [107]:
recession_start_str = recession_info['start'][1]
print(type(recession_start_str))
year,month = recession_start_str.strftime('%Y-%m').split('-')

recession_start_str = '{}q{}'.format(year, int(int(month)/3))
recession_start_str

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


'2006q4'

'''
year_begin = pd.Timestamp('1996-04')
year_end = pd.Timestamp('2016-8')
print(str(year_begin) + ' to ' + str(year_end))
month_delta = pd.Timedelta('1M')
print('delta time is ' + str(month_delta))'''
'''prng = pd.period_range('1996-04', '2016-06', freq='3M')
for month in prng:
    print(month.to_timestamp())
    month+2 month+1 and get sum of each column and get mean of three month
    if month gdp is not exist:
        skip'''

In [108]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recession_start_str = recession_info['start'][1]
    print(type(recession_start_str))
    year,month = recession_start_str.strftime('%Y-%m').split('-')

    recession_start_str = '{}q{}'.format(year, int(int(month)/3))
    return recession_start_str

In [109]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    recession_end_str = recession_info['end'][1]
    print(type(recession_end_str))
    year,month = recession_end_str.strftime('%Y-%m').split('-')

    recession_end_str = '{}q{}'.format(year, int(int(month)/3))
    return recession_end_str

In [110]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession_bottom_str = recession_info['end'][1]
    print(type(recession_bottom_str))
    year,month = recession_bottom_str.strftime('%Y-%m').split('-')

    recession_bottom_str = '{}q{}'.format(year, int(int(month)/3))
    return recession_bottom_str

In [141]:
def map_column_to_quarter(date):
    year,month = date.strftime('%Y-%m').split('-')

    return_str = '{}q{}'.format(year, int(int(month)/3))
    return return_str
    
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    convert_ct_home_price_df = get_city_homeprice()
    #print(list(ct_home_price_df.columns))
    convert_gdp_df = convert_ct_home_price_df.iloc[:, 6:]
    #gdp_df.head()
    # gdp_df.info()
    convert_gdp_df.columns = pd.DatetimeIndex(convert_gdp_df.columns)
    #gdp_df.head()
    convert_gdp_df_Q = convert_gdp_df.resample('Q', axis=1).mean()
    convert_gdp_df_Q.fillna(method = 'bfill', inplace=True, axis = 1)

    convert_gdp_df_Q.columns = convert_gdp_df_Q.columns.map(map_column_to_quarter)
    
    convert_gdp_df_Q = pd.merge(convert_ct_home_price_df.iloc[:, 1:6], convert_gdp_df_Q, how='inner', left_index=True, right_index=True)
    
    
    # https://stackoverflow.com/questions/53222171/multi-index-would-not-group-the-same-key-in-python
    # I can't group it here
    #convert_gdp_df_Q.set_index([convert_gdp_df_Q['State'], convert_gdp_df_Q['RegionName']], drop=True, inplace=True)
    
    m_index = pd.MultiIndex.from_arrays([convert_gdp_df_Q['State'], convert_gdp_df_Q['RegionName']])
    convert_gdp_df_Q.index = m_index

    return convert_gdp_df_Q

df = convert_housing_data_to_quarters()
#gdp_df_Q.head()
#df = pd.merge(ct_home_price_df.iloc[:, 1:6], gdp_df_Q,  how='inner', left_index=True, right_index=True)
df.iloc[1:10,:5]

,,RegionName,State,Metro,CountyName,SizeRank
State,RegionName,,,,,
CA,Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2
IL,Chicago,Chicago,IL,Chicago,Cook,3
PA,Philadelphia,Philadelphia,PA,Philadelphia,Philadelphia,4
AZ,Phoenix,Phoenix,AZ,Phoenix,Maricopa,5
NV,Las Vegas,Las Vegas,NV,Las Vegas,Clark,6
CA,San Diego,San Diego,CA,San Diego,San Diego,7
TX,Dallas,Dallas,TX,Dallas-Fort Worth,Dallas,8
CA,San Jose,San Jose,CA,San Jose,Santa Clara,9
FL,Jacksonville,Jacksonville,FL,Jacksonville,Duval,10


In [112]:
gdp_df_Q.set_index

,1996q2,1996q3,1996q4,1997q1,1997q2,1997q3,1997q4,1998q1,1998q2,1998q3,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,435650.000000,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,154666.666667,154200.000000,154433.333333,156866.666667,158533.333333,159266.666667,162000.000000,166966.666667,171400.000000,175966.666667,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,109466.666667,109133.333333,109600.000000,111266.666667,112200.000000,112966.666667,114833.333333,116966.666667,118433.333333,120700.000000,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,49833.333333,49366.666667,49466.666667,49600.000000,49733.333333,49733.333333,49866.666667,50300.000000,50800.000000,50666.666667,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,87700.000000,88600.000000,89733.333333,91266.666667,92033.333333,93000.000000,94633.333333,96733.333333,98433.333333,100433.333333,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [113]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"